# Get all funds

In [151]:
import tushare as ts
import pandas as pd
from math import isnan
pro = ts.pro_api('5d2dd2c56bb822ac0e818aaa4b0b344f95c7d5e7b9c83c69e2ed90ff')

Get ETF(场内基金) and LOF(场外基金), then contatenate then.

In [137]:
fund_list_O = pro.fund_basic(market = 'O') # 场外基金
fund_list_E = pro.fund_basic(market = 'E') # 场内基金

# merge the 2 fund table to get a whole perspect
fund_list = pd.concat([fund_list_E, fund_list_O], ignore_index=True) 

Drop some invalid data and useless columns.

We define invalid fund as 
- fund with 'found_date' > 20180101
- fund with 'delist_date' < 20201231

In [140]:
# Delete funds with found_date after 2018.01.01
fund_list = fund_list.drop(fund_list[(fund_list['found_date'] > '20180101') | (fund_list['delist_date'] < '20201231')].index) 

# Delete 预期收益率, 业绩比较基准, 管理费, 托管费, 起点金额(万元), 日常申购起始日, 日常赎回起始日
fund_list = fund_list.drop(columns=['exp_return', 'benchmark', 'm_fee', 'c_fee', 'min_amount', 'purc_startdate', 'redm_startdate'])

# Delete some dates, this can be revealed in funds' semi-year reports, also portfolios.
fund_list = fund_list.drop(columns=['found_date', 'due_date', 'list_date', 'issue_date', 'delist_date']) 

# Delete useless columns
fund_list = fund_list.drop(columns=['type', 'trustee', 'custodian', 'management', 'invest_type', 'duration_year'])

# Delete funds with 'issue_amount' == null
fund_list = fund_list.drop(fund_list[pd.isnull(fund_list['issue_amount'])].index)

Store the funds into `fund_list.csv`.

In [145]:
fund_list = fund_list.reset_index(drop=True)
fund_list.to_csv("data/fund_list.csv")